# This is a collection of Python snippets

## Working with Widgets in Notebooks

https://medium.com/@jdchipox/how-to-interact-with-jupyter-33a98686f24e

### Containers

In [6]:
# some handy functions to use along widgets
from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets# defining some widgets
text = widgets.Text(
       value='My Text',
       description='Title', )

calendar = widgets.DatePicker(
           description='Select Date')

slider = widgets.FloatSlider(
         value=1,
         min=0,
         max=10.0,
         step=0.1,)

menu = widgets.Dropdown(
       options=['red', 'blue', 'green'],
       value='red',
       description='Color:')

checkbox = widgets.Checkbox(
           description='Check to invert',)

In [7]:
# You can display any of them by calling the object on a cell
calendar

DatePicker(value=None, description='Select Date')

In [8]:
# Use a container widget to display individual elements together. They can be nested inside the other

box = widgets.VBox([text, slider, menu, calendar, checkbox ])
box

widgets.HBox([box, box])

### A Tabbed container

In [9]:
# A Tab container

# defining a list with the contents of our windows
children = [box, box]# initializing a tab
tab = widgets.Tab()
# setting the tab windows 
tab.children = children# changing the title of the first and second window
tab.set_title(0, 'box')
tab.set_title(1, 'copy of box')

tab

In [10]:
# Inspecting and using values of widgets

# current value of text, checkbox and calendar 
print('Widget text has value {}, of {}'.format(text.value, type(text.value)))

print('Widget checkbox has value {}, of {}'.format(checkbox.value, type(checkbox.value)))

print('Widget calendar has value {}, of {}'.format(calendar.value, type(calendar.value)))

Widget text has value My Text, of <class 'str'>
Widget checkbox has value False, of <class 'bool'>
Widget calendar has value None, of <class 'NoneType'>


### Dataset explorer and printing values

In [ ]:
# Exploring a dataset

import pandas as pd
import math
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
df = pd.read_csv('https://raw.githubusercontent.com/diegopenilla/Some_Data/master/Downloads/trainingData.csv')



# DROPDOWN MENU FOR SELECTING USER-ID
users = widgets.Dropdown(
    options=list(set(df['USERID'])),
    value=1,
    description='USER-ID:',
    disabled=False)

dicti = {} # Initialize dictionary for holding {building(s): floor(s)} for each user.

# Button to display info about user
buttonuser_info = widgets.Button(description='Show Info')
outuser_info = widgets.Output()
def on_buttonuser_info_clicked(b):
    with outuser_info:
        clear_output()
        selection = df[df['USERID']==users.value]
        visited_places = len(set(selection['SPACEID']))
        dicti = {i: 0 for i in list(set(selection['BUILDINGID']))}
        for key, val in dicti.items():
            dicti[key] = list(set(df.loc[(df["USERID"] ==users.value) & (df["BUILDINGID"] == key), "FLOOR"]))
        
        # Extract distance travelled 
        lat = list(selection["LATITUDE"])
        long = list(selection["LONGITUDE"])
        
        # Calculate distance between one point and the next
        total = []
        for latitude, longitude in zip(lat, long):
            total.append((latitude, longitude))
        rez = []
        
        # computing distance as the distance between the coordinates x,y from one point to the next.
        # (we sorted our dataframe by time, and now we are accesing data from a user)
        for i in range(len(total)-1):
            rez.append(math.hypot(total[i+1][0] - total[i][0], total[i+1][1] - total[i][1]))   
        distance_travelled = sum(rez)
        
        # display, the information in Markdown to make it look better.
        display(Markdown('''For User {}, there are ${}$ observations from ${}$ WAP (*Wireless Access Points*)
- Beginning in {} and ending in {}
- Using smartphone {}
- The user travelled {:.2f} $m$ (does not take into account vertical movemement, change of floors)
- Visited places {}
- In {} locations with different SPACEID'''.format(users.value, selection.shape[0], selection.shape[1]-9,         
                                                   str(selection['TIMESTAMP'].min()), str(selection['TIMESTAMP'].max()),
                                                   str(list(set(selection['PHONEID']))[0]), distance_travelled,
                                                   str(dicti),len(set(selection['SPACEID'])))))

# Links button: buttonuser_info to its output function      
buttonuser_info.on_click(on_buttonuser_info_clicked)

# Display
display(widgets.VBox([users,buttonuser_info, outuser_info]))

### Filter a dataframe by value in a column

In [17]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# read in the csv file
df = pd.read_csv('pivot_raw_data.csv')

df.head()

,Date_of_Week,Year,Month,Channel,Brand,Country,Category,Partnership,NMV,Item,ASP,PV
0,18/2/18,2018,Feb,LLLDD,AV,TH,Health & Beauty,MP,18853.22964,1243.42604,15.162325,41129
1,28/1/18,2018,Jan,LLLDD,AV,SG,Health & Beauty,MP,0.00000,0.00000,NaN,440
2,14/1/18,2018,Jan,LLLDD,AV,TH,Health & Beauty,MP,17608.25665,1213.82701,14.506397,43215
3,4/2/18,2018,Feb,LLLDD,AV,SG,Health & Beauty,MP,0.00000,0.00000,NaN,564
4,21/1/18,2018,Jan,LLLDD,AV,TH,Health & Beauty,MP,18179.10355,1245.01100,14.601561,44818


In [ ]:
@interact
def show_nmv_more_than(column=['ASP', 'NMV', 'PV', 'Item'], 
                            x=(10, 100000, 20000)):
    return df.loc[df[column] > x]

### Widgets for Plots